<h1><center>Universidade Federal de Minas Gerais</center></h1>
<h3><center>Departamento de Ciência da Computação</center></h3>
<h3><center>Introdução a Banco de Dados -  Trabalho Prático #1</center></h3>


# 1. Introdução

### 1.1. Contexto

Nesse trabalho prático utilizaremos o banco de dados representado pelo schema abaixo:

![Modelo_BD.png](https://github.com/claudiovaliense/ibd/blob/main/data/schema.png?raw=true)

### 1.2. Entrega




1.   Você deve completar esse notebook, executar todas as células e submetê-lo no **Minha UFMG**.
2.   Você pode entregar até o dia 23/01/2022 às 23:59.
3. O trabalho é individual e vale 15 pontos.






# 2. Preparação do Ambiente

In [2]:
# importando os packages necessários
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
import wget

In [2]:
wget.download("https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql")


  0% [                                                                            ]      0 / 277046
  2% [..                                                                          ]   8192 / 277046
  5% [....                                                                        ]  16384 / 277046
  8% [......                                                                      ]  24576 / 277046
 11% [........                                                                    ]  32768 / 277046
 14% [...........                                                                 ]  40960 / 277046
 17% [.............                                                               ]  49152 / 277046
 20% [...............                                                             ]  57344 / 277046
 23% [.................                                                           ]  65536 / 277046
 26% [....................                                                        ]  73728 / 277046

'despesas_publicas_tp1.sql'

In [3]:
# download do dump do banco de  dados:
!wget https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql

'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


### 2.1. Criação da conexão com o banco
* No momento da conexão, é informado o nome do arquivo onde será montado o banco de dados. No nosso caso, este arquivo não precisa ser pré-existente, porque iremos carregar as informações de um dump;
* Se este comando for executado com um arquivo que já existe e possui as mesmas tabelas que serão criadas, ocorrerá um erro no momento da execução do script mais à frente.

In [3]:
conn = sqlite3.connect('despesas.db')
cursor = conn.cursor()

### 2.2. Execução do script para inicializar e popular o banco de dados

* Criada a conexão, iremos utilizar o arquivo .sql disponibilizado para recuperar o schema e os dados do banco

In [3]:
f = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
sql = f.read()
cursor.executescript(sql)

OperationalError: table [ORGAO] already exists

### 2.3. Realizando consultas no banco de dados

* Criada a conexão e realizada a carga dos dados no banco, pode ser executada qualquer consulta SQL;
* O SQLite3 permite que o retorno dos dados seja feito em vários formatos, entretanto, utilizaremos o retorno em um dataframe do Pandas.



**CONSULTA EXEMPLO:** todos os órgãos ordenados pela quantidade de orgãos subordinados de forma decrescente.

In [4]:
df = pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,26000,Ministério da Educação,104
1,30000,Ministério da Justiça e Segurança Pública,6
2,22000,"Ministério da Agricultura, Pecuária e Abastec",5
3,24000,"Ministério da Ciência, Tecnologia, Inovações",4
4,52000,Ministério da Defesa,4
5,53000,Ministério da Integração Nacional,4
6,25000,Ministério da Fazenda,3
7,36000,Ministério da Saúde,3
8,20000,Presidência da República,2
9,39000,"Ministério dos Transportes, Portos e Aviação",2


#3. Consultas

Expresse cada consulta abaixo utilizando SQL: 


**1.** Retorne a quantidade de palavras em cada registro da coluna nome_orgao da tabela orgão.

In [7]:
q1 = pd.read_sql_query(
  "SELECT LENGTH(LTRIM(RTRIM(nome_orgao))) - LENGTH(REPLACE(LTRIM(RTRIM(nome_orgao)), ' ', '')) + 1 " + 
  "AS QUANTIDADE_PALAVRAS " +
  "FROM orgao"
, conn)

q1

,QUANTIDADE_PALAVRAS
0,3
1,6
2,5
3,3
4,3
5,6
6,4
7,3
8,6
9,4




```
# Isto está formatado como código
```

**2.** Dado a coluna nome_orgao, retorne a primeira posição que a string 'da' aparece da tabela orgão 

In [5]:
q2 = pd.read_sql_query("SELECT INSTR(nome_orgao, 'da') FROM orgao", conn)

q2

,"INSTR(nome_orgao, 'da')"
0,13
1,12
2,12
3,12
4,12
5,12
6,12
7,12
8,0
9,0


**3.** Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [6]:
q3 = pd.read_sql_query(
  "SELECT o.nome_orgao, sum(dp.valor_despesa) AS DESPESAS " + 
  "FROM orgao AS o, despesa_publica AS dp " + 
  "WHERE o.cod_orgao = dp.cod_orgao " +
  "AND dp.ano_despesa = 2018 " +
  "GROUP BY dp.cod_orgao " + 
  "ORDER BY despesas DESC", conn)
  
q3

,NOME_ORGAO,DESPESAS
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,Ministério dos Direitos Humanos,0.00
6,Ministério da Integração Nacional,0.00
7,Ministério do Meio Ambiente,0.00
8,"Ministério dos Transportes, Portos e Aviação",0.00
9,Ministério das Relações Exteriores,0.00


**4.** Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.

> Bloco com recuo



In [7]:
q4 = pd.read_sql_query(
  "SELECT dp.mes_despesa, dp.ano_despesa, dp.modalidade_despesa, pg.nome_programa_governo, dp.valor_despesa " + 
  "FROM orgao AS o, despesa_publica AS dp, programa_governo AS pg " + 
  "WHERE o.cod_orgao = dp.cod_orgao AND dp.cod_programa_governo = pg.cod_programa_governo " +
  "AND o.nome_orgao = 'Ministério da Fazenda' " +
  "AND mes_despesa <= 6"
  , conn)
  
q4

,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,4,2018,Reserva de Contingência,GESTAO AEROPORTUARIA,0
1,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
3,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
4,4,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
5,5,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
6,6,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


**5.** Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [8]:
q5 = pd.read_sql_query(
  "SELECT pg.cod_programa_governo, pg.nome_programa_governo " + 
  "FROM programa_governo AS pg, despesa_publica AS dp, detalhamento_funcao AS df, funcao AS f " + 
  "WHERE pg.cod_programa_governo = dp.cod_programa_governo AND dp.cod_detalhamento = df.cod_detalhamento AND df.cod_funcao = f.cod_funcao " +
  "AND f.nome_funcao = 'Educação' " +
  "GROUP BY pg.cod_programa_governo " +
  "ORDER BY sum(dp.valor_despesa) DESC " +
  "LIMIT 3"
, conn)

q5

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


**6.** Transforme os caracteres em minúsculo e remova todos os termos 'da' na coluna nome_orgao da tabela orgão.

In [9]:
q6 = pd.read_sql_query(
  "SELECT LOWER(REPLACE(nome_orgao, ' da ', ' ')) AS nome_orgao " + 
  "FROM orgao" 
, conn)

# q6 = pd.read_sql_query(
#   "UPDATE orgao " +
#   "SET nome_orgao = LOWER(REPLACE(nome_orgao, ' da ', ' '))"
# , conn)

q6

,nome_orgao
0,presidência república
1,"ministério agricultura, pecuária e abastec"
2,"ministério ciência, tecnologia, inovações"
3,ministério fazenda
4,ministério educação
5,ministério justiça e segurança pública
6,ministério das relações exteriores
7,ministério saúde
8,"ministério dos transportes, portos e aviação"
9,ministério do meio ambiente


**7.** Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

In [10]:
q7 = pd.read_sql_query(
  "SELECT o.nome_orgao, COUNT(DISTINCT pg.cod_programa_governo) AS QUANTIDADE_PROGRAMAS " + 
  "FROM orgao AS o, programa_governo AS pg, despesa_publica AS dp " +
  "WHERE o.cod_orgao = dp.cod_orgao AND dp.cod_programa_governo = pg.cod_programa_governo " +
  "AND dp.valor_despesa > 0 " +
  "GROUP BY o.cod_orgao "
, conn)

q7

,NOME_ORGAO,QUANTIDADE_PROGRAMAS
0,"Ministério da Agricultura, Pecuária e Abastec",2
1,Ministério da Educação,4
2,Ministério da Saúde,2
3,Ministério da Defesa,2
4,Ministério do Desenvolvimento Social,1


**8.** Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [11]:
q8 = pd.read_sql_query(
  "SELECT pg.nome_programa_governo, sf.nome_subfuncao, COUNT(dp.cod_despesa) AS QT_DESPESAS_PUBLICAS, SUM(dp.valor_despesa) AS TOTAL_DESPESAS " + 
  "FROM subfuncao AS sf, despesa_publica AS dp, programa_governo AS pg, detalhamento_funcao AS df "
  "WHERE pg.cod_programa_governo = dp.cod_programa_governo AND dp.cod_detalhamento = df.cod_detalhamento AND df.cod_subfuncao = sf.cod_subfuncao " +
  "GROUP BY pg.cod_programa_governo"
, conn)

q8

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,QT_DESPESAS_PUBLICAS,TOTAL_DESPESAS
0,VIVER SEM LIMITE,Normatização e fiscalização,150,0.00
1,SOS EMERGENCIA,Assistência hospitalar e ambulatorial,3,0.00
2,MELHOR EM CASA,Atenção básica,16,0.00
3,UBS,Atenção básica,9,0.00
4,"UPA (INVESTIMENTO, CUSTEIO E COMPLEXO DE REGUL...",Assistência hospitalar e ambulatorial,12,0.00
5,REDE CEGONHA,Assistência hospitalar e ambulatorial,38,1210.00
6,CONTROLE DO CANCER,Assistência hospitalar e ambulatorial,40,38971.19
7,CIENCIA SEM FRONTEIRAS,Ensino superior,24,0.00
8,CRIMINALIDADE VIOLENTA,Policiamento,1,0.00
9,ENEM,Educação básica,53,334262.06


**9.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

In [12]:
q9 = pd.read_sql_query(
  "SELECT dp.cod_orgao, dp.cod_despesa, dp.mes_despesa, dp.ano_despesa, dp.modalidade_despesa, dp.valor_despesa " +
  "FROM despesa_publica AS dp " +
  "WHERE dp.cod_orgao = " +
  "(" +
    "SELECT o.cod_orgao " +
    "FROM orgao AS o " +
    "ORDER BY o.qtd_orgaos_subordinados DESC " +
    "LIMIT 1" +
  ")"
, conn)

q9

,COD_ORGAO,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,26000,196,10,2018,Reserva de Contingência,0.0
1,26000,197,1,2018,Reserva de Contingência,0.0
2,26000,198,2,2018,Reserva de Contingência,0.0
3,26000,199,3,2018,Reserva de Contingência,0.0
4,26000,200,4,2018,Reserva de Contingência,0.0
...,...,...,...,...,...,...
289,26000,485,8,2018,Reserva de Contingência,2400.0
290,26000,486,9,2018,Reserva de Contingência,0.0
291,26000,487,10,2018,Reserva de Contingência,0.0
292,26000,488,11,2018,Reserva de Contingência,0.0


**10.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior valor total em despesas durante o ano de 2018.

In [13]:
q10 = pd.read_sql_query(
  "SELECT dp.cod_despesa, dp.mes_despesa, dp.ano_despesa, dp.modalidade_despesa, dp.valor_despesa " +
  "FROM despesa_publica AS dp " +
  "WHERE dp.cod_orgao = "+
  "(" +
    "SELECT o.cod_orgao " + 
    "FROM orgao AS o, despesa_publica AS dp " + 
    "WHERE o.cod_orgao = dp.cod_orgao AND dp.ano_despesa = 2018 " +
    "GROUP BY o.cod_orgao " +
    "ORDER BY (SUM(dp.valor_despesa)) DESC " + 
    "LIMIT 1" +
  ")"
, conn)
q10

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,827,1,2018,Reserva de Contingência,0.0
1,828,2,2018,Reserva de Contingência,0.0
2,829,3,2018,Reserva de Contingência,0.0
3,830,4,2018,Reserva de Contingência,0.0
4,831,5,2018,Reserva de Contingência,0.0
5,832,6,2018,Reserva de Contingência,0.0
6,833,7,2018,Reserva de Contingência,0.0
7,834,8,2018,Reserva de Contingência,0.0
8,835,9,2018,Reserva de Contingência,334126.7
9,836,10,2018,Reserva de Contingência,0.0


**11.** Liste os códigos e nomes dos programas de governo relacionados ao órgão (ou órgãos, caso haja empate) que teve menos registros de despesas públicas durante o ano.

In [16]:

q11 = pd.read_sql_query(
  "SELECT DISTINCT pg.cod_programa_governo, pg.nome_programa_governo " +
  "FROM despesa_publica AS dp, programa_governo as pg " +
  "WHERE dp.cod_programa_governo = pg.cod_programa_governo " +
  "AND dp.cod_orgao IN " +
  "(" +
    "SELECT o.cod_orgao " +
    "FROM orgao AS o, despesa_publica AS dp " +
    "WHERE o.cod_orgao = dp.cod_orgao " +
    "GROUP BY o.cod_orgao " +
    "HAVING COUNT(dp.cod_despesa) = " +
    "(" +
      "SELECT MIN(qtd_despesas) FROM " +
      "(" +
        "SELECT COUNT(dp.cod_despesa) AS qtd_despesas " +
        "FROM orgao AS o, despesa_publica AS dp " +
        "WHERE o.cod_orgao = dp.cod_orgao " +
        "GROUP BY o.cod_orgao" +
      ")" +
    ")" +
  ")"
, conn)

q11

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,19,OLIMPIADAS
1,24,BRASIL SEM MISERIA


**12.**  Álgebra relacional: projeção do NOME_FUNCAO e NOME_SUBFUNCAO da junção natural das tabelas FUNCAO, DETALHAMENTO_FUNCAO e SUBFUNCAO

In [17]:
"""
A <- DETALHAMENTO_FUNCAO ⋈ FUNCAO
B <- A ⋈ SUBFUNCAO
RESULT <- Πnome_funcao,nome_subfuncao(B)
"""


q12 = pd.read_sql_query(
  "SELECT DISTINCT f.nome_funcao, sb.nome_subfuncao " +
  "FROM detalhamento_funcao " +
  "NATURAL JOIN funcao AS f " +
  "NATURAL JOIN subfuncao AS sb"
, conn)

q12

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


**13.**  Álgebra relacional: projeção de MODALIDADE_DESPESA, VALOR_DESPESA e NOME_PROGRAMA_GOVERNO da junção natural das tabelas PROGRAMA_GOVERNO e DESPESA_PUBLICA onde a MODALIDADE_DESPESA = 'Reserva de Contingência'

In [18]:
"""
A <- σmodalidade_despesa='Reserva de Contingência'(DESPESA_PUBLICA)
B <- A ⋈ PROGRAMA_GOVERNO
RESULT <- Πmodalidade_despesa,valor_despesa,nome_programa_governo(B)
"""

q13 = pd.read_sql_query(
  "SELECT DISTINCT dp.modalidade_despesa, dp.valor_despesa, pg.nome_programa_governo " +
  "FROM despesa_publica AS dp " +
  "NATURAL JOIN programa_governo AS pg " +
  "WHERE dp.modalidade_despesa = 'Reserva de Contingência'"
, conn)

q13

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.00,BRASIL SEM MISERIA
1,Reserva de Contingência,0.00,VIOLENCIA NO TRANSITO
2,Reserva de Contingência,0.00,TERRITORIOS DA CIDADANIA
3,Reserva de Contingência,8215.70,BRASIL SEM MISERIA
4,Reserva de Contingência,3.42,BRASIL SEM MISERIA
5,Reserva de Contingência,81120.13,BRASIL SEM MISERIA
6,Reserva de Contingência,0.00,REGULARIZACAO FUNDIARIA
7,Reserva de Contingência,1497.26,REGULARIZACAO FUNDIARIA
8,Reserva de Contingência,0.00,SEGURO RURAL PRIVADO
9,Reserva de Contingência,0.00,VIVER SEM LIMITE


**14.**  Álgebra relacional: projeção de VALOR_DESPESA e NOME_ORGAO da junção natural das tabelas DESPESA_PUBLICA e ORGAO onde o NOME_ORGAO = 'Ministério da Fazenda'

In [19]:
"""
A <- σnome_orgao='Ministério da Fazenda'(ORGAO)
B <- A ⋈ DESPESA_PUBLICA
RESULT <- Πvalor_despesa,nome_orgao(B)
"""

q14 = pd.read_sql_query(
  "SELECT DISTINCT dp.valor_despesa, o.nome_orgao " +
  "FROM orgao AS o " +
  "NATURAL JOIN despesa_publica AS dp " +
  "WHERE o.nome_orgao = 'Ministério da Fazenda'"
, conn)

q14

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda


**15.**  Álgebra relacional: projeção de NOME_PROGRAMA_GOVERNO e VALOR_DESPESA da junção natural das tabelas DESPESA_PUBLICA, PROGRAMA_GOVERNO e ORGAO onde o NOME_ORGAO = 'Ministério da Saúde'

In [20]:
"""
A <- σnome_orgao='Ministério da Saúde'(ORGAO)
B <- A ⋈ DESPESA_PUBLICA
C <- B ⋈ PROGRAMA_GOVERNO
RESULT <- Πnome_programa_governo,valor_despesa(C)
"""
q15 = pd.read_sql_query(
  "SELECT DISTINCT pg.nome_programa_governo, dp.valor_despesa " +
  "FROM orgao AS o " +
  "NATURAL JOIN despesa_publica AS dp " +
  "NATURAL JOIN programa_governo AS pg " +
  "WHERE o.nome_orgao = 'Ministério da Saúde'"
, conn)

q15

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.00
1,SOS EMERGENCIA,0.00
2,MELHOR EM CASA,0.00
3,UBS,0.00
4,"UPA (INVESTIMENTO, CUSTEIO E COMPLEXO DE REGUL...",0.00
5,REDE CEGONHA,0.00
6,CONTROLE DO CANCER,0.00
7,CONTROLE DO CANCER,38971.19
8,"CRACK, E POSSIVEL VENCER",0.00
9,BRASIL SEM MISERIA,0.00


# 4. Fechamento da conexão com o banco

* Após serem realizadas todas as consultas necessárias, é aconselhavel encerrar formalmente a conexão com o banco de dados

In [ ]:
conn.close()